In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
dataset = pd.read_csv("loan_approval_dataset.csv")
dataset.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [5]:
dataset.shape

(4269, 13)

In [6]:
dataset.nunique()

,0
loan_id,4269
no_of_dependents,6
education,2
self_employed,2
income_annum,98
loan_amount,378
loan_term,10
cibil_score,601
residential_assets_value,278
commercial_assets_value,188


In [7]:
#duplicates
dataset.duplicated().sum()

0

In [8]:
dataset.drop(columns=['loan_id'], inplace=True)

In [9]:
dataset.columns

Index([' no_of_dependents', ' education', ' self_employed', ' income_annum',
       ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')

In [10]:
dataset.columns = [c.strip() for c in dataset.columns]
dataset.columns

Index(['no_of_dependents', 'education', 'self_employed', 'income_annum',
       'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value',
       'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value',
       'loan_status'],
      dtype='object')

In [11]:
X = dataset.drop(columns=['loan_status'])
y = dataset['loan_status']

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   no_of_dependents          4269 non-null   int64 
 1   education                 4269 non-null   object
 2   self_employed             4269 non-null   object
 3   income_annum              4269 non-null   int64 
 4   loan_amount               4269 non-null   int64 
 5   loan_term                 4269 non-null   int64 
 6   cibil_score               4269 non-null   int64 
 7   residential_assets_value  4269 non-null   int64 
 8   commercial_assets_value   4269 non-null   int64 
 9   luxury_assets_value       4269 non-null   int64 
 10  bank_asset_value          4269 non-null   int64 
dtypes: int64(9), object(2)
memory usage: 367.0+ KB


In [13]:
# Apply the domain knowledge as part of preprocessing
X['total_assets_value'] = X['residential_assets_value'] + X['commercial_assets_value'] + X['luxury_assets_value'] + X['bank_asset_value']

# drop the columns
X.drop(columns=['residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value'], inplace=True)

In [14]:
X.head()

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,total_assets_value
0,2,Graduate,No,9600000,29900000,12,778,50700000
1,0,Not Graduate,Yes,4100000,12200000,8,417,17000000
2,3,Graduate,No,9100000,29700000,20,506,57700000
3,3,Graduate,No,8200000,30700000,8,467,52700000
4,5,Not Graduate,Yes,9800000,24200000,20,382,55000000


In [15]:
X['education'].unique()

array([' Graduate', ' Not Graduate'], dtype=object)

In [16]:
X['self_employed'].unique()

array([' No', ' Yes'], dtype=object)

In [17]:
y.unique()

array([' Approved', ' Rejected'], dtype=object)

In [18]:
X['education'] = X['education'].str.strip()
X['self_employed'] = X['self_employed'].str.strip()
y = y.str.strip()

In [19]:
X['self_employed'].unique()

array(['No', 'Yes'], dtype=object)

In [20]:
# convert the above columns into numerical format
# Transformation to Binary
def transform_category_to_binary(df, columns_to_transform):
  """
  Transforms specified categorical columns in a DataFrame to binary (0 and 1).

  Args:
    df: The DataFrame to modify.
    columns_to_transform: A dictionary where keys are
    column names and values are lists of values to transform to 1.

  Returns:
    The modified DataFrame with Transformed columns.
  """

  for column_name, positive_values in columns_to_transform.items():
    df[column_name] = df[column_name].apply(lambda x: 1 if x in positive_values else 0)

  return df

columns_to_transform = {
    'education': ['Graduate'],
    'self_employed': ['Yes']
}

Transformed_X = transform_category_to_binary(X, columns_to_transform)
Transformed_X.head()

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,total_assets_value
0,2,1,0,9600000,29900000,12,778,50700000
1,0,0,1,4100000,12200000,8,417,17000000
2,3,1,0,9100000,29700000,20,506,57700000
3,3,1,0,8200000,30700000,8,467,52700000
4,5,0,1,9800000,24200000,20,382,55000000


In [21]:
# log transformation
log_cols = ['income_annum', 'loan_amount', 'total_assets_value']
X[log_cols] = np.log(X[log_cols])

In [22]:
y = y.map({'Approved': 1, 'Rejected': 0})

In [23]:
y[:5]

,loan_status
0,1
1,0
2,0
3,0
4,0


In [24]:
X.head()

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,total_assets_value
0,2,1,0,16.077274,17.213369,12,778,17.741436
1,0,0,1,15.226498,16.316947,8,417,16.648724
2,3,1,0,16.023785,17.206658,20,506,17.870768
3,3,1,0,15.919645,17.239773,8,467,17.780126
4,5,0,1,16.097893,17.001863,20,382,17.822844


In [25]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
# Building the Model
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
y_pred_test = log.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred_test)
print(f'Accuracy: {acc}')

Accuracy: 0.9008587041373927


# Serialization and Deserialization

In [29]:
# serialization & deserialization
import joblib
joblib.dump(log, 'my_trained_model_v1.pkl')

['my_trained_model_v1.pkl']

In [30]:
#deserialization
final_model = joblib.load('my_trained_model_v1.pkl')

In [31]:
final_model.intercept_, final_model.coef_

(array([-1.13026847]),
 array([[-0.03113815,  0.00348628,  0.08350233, -1.23533938,  1.40374087,
         -0.13433325,  0.02375754, -0.81755529]]))

In [32]:
log.intercept_, log.coef_

(array([-1.13026847]),
 array([[-0.03113815,  0.00348628,  0.08350233, -1.23533938,  1.40374087,
         -0.13433325,  0.02375754, -0.81755529]]))

# Package and Modules

In [33]:
import PackageA

In [34]:
from PackageA import f1

In [35]:
f1.print_something()

'output from f1'

In [36]:
from PackageA import f2

In [37]:
f2.print_something()

'output from f2'

In [38]:
from PackageA.f1 import print_something as f1p

In [39]:
f1p()

'output from f1'

In [41]:
from PackageA.SubPackageA import f3

In [42]:
f3.print_something()

'output from f3'

In [43]:
from PackageA.SubPackageB import f5
f5.print_something()

'output from f5'

In [44]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor',
 '/root/.ipython']

In [45]:
sys.path.append('/your/path')